In [1]:
import os
import sys
import csv
import shutil
import random
import glob
import pandas as pd
import numpy as np
from scipy import stats
import subprocess
from multiprocessing import Pool

In [8]:
result_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_txt"
m6a_dir = "/home/galaxy/project/QTL_analysis/data/m6a_peak"
# m6a_list = glob.glob("%s/*.bed" % m6a_dir)
qtl_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/GRCh38_bed"
qtl_list = glob.glob("%s/*.bed" % qtl_dir)
postfix = ".v7.egenes_position.bed"

In [9]:
def generate_map_dict():
    map_dict = {"Adrenal": "Kidney", "Brain": "Brain", "Heart": "Heart", "Liver": "Liver", "Lung": "Lung", 
                "Muscle": "Muscle", "Pituitary": "Brain", "Stomach": "Stomach"}
    return map_dict

def perform_intersection(eQTL, m6a):
    pre_eQTL, pre_m6a = os.path.basename(eQTL).split(postfix)[0], os.path.basename(m6a).split(".bed")[0]
    result_txt = os.path.join(result_dir, "%s-%s" % (pre_eQTL, os.path.basename(m6a).replace(".bed", ".txt")))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (eQTL, m6a, result_txt))

### Perform intersection

In [10]:
map_dict = generate_map_dict()
for qtl in qtl_list:
    prefix = os.path.basename(qtl).split(".")[0].split("_")[0]
    if prefix in map_dict:
        m6a = "%s/%s.bed" % (m6a_dir, map_dict[prefix])
        print(os.path.basename(qtl), "\t", os.path.basename(m6a))
        perform_intersection(qtl, m6a)

Brain_Nucleus_accumbens_basal_ganglia.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Hypothalamus.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Pituitary.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Hippocampus.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Adrenal_Gland.v7.signif_variant_gene_pairs.bed 	 Kidney.bed
Brain_Caudate_basal_ganglia.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Anterior_cingulate_cortex_BA24.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Substantia_nigra.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Muscle_Skeletal.v7.signif_variant_gene_pairs.bed 	 Muscle.bed
Brain_Putamen_basal_ganglia.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Frontal_Cortex_BA9.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Brain_Spinal_cord_cervical_c-1.v7.signif_variant_gene_pairs.bed 	 Brain.bed
Heart_Left_Ventricle.v7.signif_variant_gene_pairs.bed 	 Heart.bed
Heart_Atrial_Appendage.v7.signif_variant_gene_pairs.bed 	 Heart.bed
Lung.v7.signif_variant_gene_pairs.bed

#### Format into bed

In [13]:
#########################################################################
data_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_txt"
result_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_bed"
qtl_list = glob.glob("%s/*.txt" % data_dir)
for qtl in qtl_list:
    tissue = os.path.basename(qtl).split("-")[-1].split(".")[0]
    df = pd.read_table(qtl, sep="\t", header=None)
    df["tissue"] = tissue
    df["info"]=df.iloc[:, 3]+";"+df.iloc[:, 4]+":"+df.iloc[:, 5].astype(str)+"-"+df.iloc[:, 6].astype(str)+";"+df.iloc[:, 7]
    result_bed = os.path.join(result_dir, os.path.basename(qtl).replace(".txt", ".bed"))
    df_bed = df.iloc[:, [0,1,2,-1]]
    df_bed.to_csv(result_bed, sep="\t", header=False, index=False)

###  Parse result
#### m6a-related eQTL

In [6]:
# # Format result
# data_dir = "/home/galaxy/project/QTL_analysis/intersected_with_m6a/eQTL"
# def construct_directory():
#     eQTL_result_dir = os.path.join(data_dir, "intersected_eQTL_bed")
#     m6a_result_dir = os.path.join(data_dir, "intersected_m6a_bed")
#     for result_dir in [eQTL_result_dir, m6a_result_dir]:
#         if not os.path.exists(result_dir):
#             os.makedirs(result_dir)
#     return eQTL_result_dir, m6a_result_dir
                
# def split_m6aQTL():
#     eQTL_result_dir, m6a_result_dir = construct_directory()
#     in_list = glob.glob("%s/*.bed" % data_dir)
#     for raw_file in in_list:
#         eQTL = "%s.bed" % os.path.basename(raw_file).split("-")[0]
#         os.system("cut -f 1-4 %s | sort -k1,1 -k2,2n | uniq> %s" % (raw_file, os.path.join(eQTL_result_dir, eQTL)))
#         m6a = os.path.basename(raw_file)
#         os.system("cut -f 5-7 %s | sort -k1,1 -k2,2n | uniq > %s" % (raw_file, os.path.join(m6a_result_dir, m6a)))

# # Perform
# split_m6aQTL()